In [ ]:
import numpy as np
import pandas as pd
import sqlite3
import os
import csv

In [ ]:
db_bd = sqlite3.connect("homodimer_ppi_features.sqlite")

with db_bd:
    db_bd.execute('ATTACH DATABASE "features.sqlite" AS features')
    db_bd.execute("ATTACH DATABASE 'localization_data.sqlite' AS localization")
    db_bd.execute('ATTACH DATABASE "orf_features.sqlite" AS orf_features')

In [ ]:
with db_bd:
    pd.read_csv("explained_ppis_modeling.csv").to_sql("homodimer_ppi_features",db_bd)

In [ ]:
print(pd.read_sql_query("SELECT * FROM ohnologs limit 5",db_bd))
print(pd.read_sql_query("SELECT * FROM marchant_paralogs limit 5",db_bd))
print(pd.read_sql_query("SELECT * FROM homologs limit 5",db_bd))
print(pd.read_sql_query("SELECT * FROM localization_jacard limit 5",db_bd))
print(pd.read_sql_query("SELECT * FROM per_orf limit 5",db_bd))

In [ ]:
with db_bd:
    db_bd.execute("""
        CREATE TABLE per_ppi AS
        SELECT * FROM (
            SELECT DISTINCT PPI,ORF1,ORF2,Explained,estimate,stderr,tvalue,pvalue,qvalz,r_squared,pearson 
            FROM homodimer_ppi_features
            )
        LEFT OUTER JOIN
            homologs
        USING (ORF1,ORF2)
        LEFT OUTER JOIN
            marchant_paralogs
        USING (ORF1,ORF2)
        LEFT OUTER JOIN
            ohnologs
        USING (ORF1,ORF2)
        LEFT OUTER JOIN
            localization_jacard
        USING (ORF1,ORF2)
        """)

#db_bd.execute("DROP TABLE per_ppi")

In [ ]:
print(pd.read_sql_query("SELECT * FROM per_ppi WHERE Duplication is 'SSD' limit 50",db_bd))
# So there's none from the marchant set that are SSDs, just WGDs

In [ ]:
with db_bd:
    db_bd.execute("""
        CREATE TABLE homodimer_features AS
        SELECT * FROM 
            ( SELECT * FROM per_ppi 
            LEFT OUTER JOIN 
                ( SELECT *,YORF AS ORF1 FROM per_orf )
            USING (ORF1)
            )
        UNION
        SELECT * FROM 
            ( SELECT * FROM per_ppi 
            LEFT OUTER JOIN 
                ( SELECT *,YORF AS ORF2 FROM per_orf )
            USING (ORF2)
            )
        """)

In [ ]:
print(pd.read_sql_query("SELECT * FROM homodimer_features limit 5",db_bd))

In [ ]:
with db_bd:
    db_bd.execute("\
        UPDATE homodimer_features \
            SET ohnolog = 0 \
            WHERE ohnolog IS NULL \
        ")
    db_bd.execute("\
        UPDATE homodimer_features \
            SET homolog = 0 \
            WHERE homolog IS NULL \
        ")
    db_bd.execute("\
        UPDATE homodimer_features \
            SET Colocalization = 0 \
            WHERE Colocalization IS NULL \
        ")

In [ ]:
pd.read_sql_query("\
    SELECT DISTINCT * FROM homodimer_features \
    ",db_bd).iloc[0,]

In [ ]:
db_bd.close()